In [2]:
import numpy as np
import os

In [3]:
def compute_center(box):
    return ((box[0] + box[2]) / 2, (box[1] + box[3]) / 2)

def compute_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)

    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

predicted_path = "runs/detect/predict1/labels"
true_path = "labels/valid"

distances = []
ious = []
pred_labels = []
true_labels = []

for filename in os.listdir(predicted_path):
    with open(os.path.join(predicted_path, filename), 'r') as f:
        preds_data = [line.split() for line in f.readlines()]
        preds = [list(map(float, data[1:])) for data in preds_data]
        pred_label = [int(data[0]) for data in preds_data]

    with open(os.path.join(true_path, filename), 'r') as f:
        true_data = f.readline().split()
        true = list(map(float, true_data[1:]))
        true_label = int(true_data[0])

    center_true = compute_center(true)
    min_distance = float('inf')
    best_iou = 0
    best_pred_label = None
    
    for i, pred in enumerate(preds):
        center_pred = compute_center(pred)

        distance = np.linalg.norm(np.array(center_pred) - np.array(center_true))
        if distance < min_distance:
            min_distance = distance
            best_iou = compute_iou(pred, true)
            best_pred_label = pred_label[i]

    distances.append(min_distance)
    ious.append(best_iou)
    pred_labels.append(best_pred_label)
    true_labels.append(true_label)

print(f"Mean distance: {np.mean(distances)}")
print(f"Standard Deviation of distance: {np.std(distances)}")
print(f"Mean IoU: {np.mean(ious)}")
print(f"Standard Deviation of IoU: {np.std(ious)}")



Mean distance: 0.12460187106854954
Standard Deviation of distance: 0.12068045885788577
Mean IoU: 0.7250951305519128
Standard Deviation of IoU: 0.24165147425530464


In [4]:
num_predicted_imgs = len(os.listdir(predicted_path))
num_origin_imgs = len(os.listdir(true_path))
ratio = round((num_predicted_imgs/num_origin_imgs)*100,2)
print(f"The percentage of original images which has detection is: {ratio}%")

The percentage of original images which has detection is: 83.33%


In [5]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Assuming `pred_labels` are your predicted labels and `true_labels` are the ground truth
conf_matrix = confusion_matrix(true_labels, pred_labels)
accuracy = accuracy_score(true_labels, pred_labels)
precision = precision_score(true_labels, pred_labels, average='macro')  # 'macro' for binary problem
recall = recall_score(true_labels, pred_labels, average='macro')
f1 = f1_score(true_labels, pred_labels, average='macro')

print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Accuracy: {accuracy:.2%}")
print(f"Precision: {precision:.2%}")
print(f"Recall: {recall:.2%}")
print(f"F1-Score: {f1:.2%}")


Confusion Matrix:
[[27  5]
 [ 3 25]]
Accuracy: 86.67%
Precision: 86.67%
Recall: 86.83%
F1-Score: 86.65%
